In [1]:
import pandas as pd 
import numpy as np 
import os
import json
import re

In [2]:
def extractLatLong(row):
    latLong = row["latLong"]
    if latLong == "{}" or len(latLong) <= 0:
        return np.nan, np.nan
    latLong = eval(latLong)
    lat = latLong["latitude"]
    long = latLong["longitude"]
    return lat, long

def extractBedPrice(row):
    if pd.notnull(row['units']):
        row['beds'] = row['units']['beds']
        row['unformattedPrice'] = row['units']['price']
        row['baths'] = 0
    return row

def process_zillow(df):
    usecols = [
        'units',
        'beds',
        'baths',
        'unformattedPrice',
        'address',
        'addressStreet',
        'addressState',
        'addressCity',
        'addressZipcode',
        'latLong'
    ]
    df = df[usecols]
    df[['lat', 'long']] = df[["latLong"]].apply(lambda x: extractLatLong(x), axis=1, result_type="expand")
    df['units'] = df['units'].apply(lambda x: np.nan if pd.isna(x) else eval(x))
    df = df.explode('units', ignore_index=True)
    df = df.apply(lambda row: extractBedPrice(row), axis=1, result_type="expand")
    df = df.drop(columns=["units", 'latLong'])
    df = df.rename(columns={
        'unformattedPrice': 'price',
        'addressStreet': 'street',
        'addressState': 'state',
        'addressCity': 'city',
        'addressZipcode': 'zipcode',
    })
    df['beds'] = df['beds'].astype(int)
    df['price'] = df['price'].apply(
        lambda x: int(x.replace('$', '').replace(',', '').replace('+', '')) if isinstance(x, str) else int(x)
    )
    return df

In [3]:
amenities_lst = ['community_amenities', 'amenities', 'Additional Upgraded and Premium Features', 'Upgraded Residence Amenities']

def parse_garden_amenity(amenities):
    return [amenity.strip() for amenity in amenities.strip().split('-') if (len(amenity.strip()) > 0) and ("offer" not in amenity.lower())]

def get_amenity_set(row):
    am, cm, adm, ura = [], [], [], []
    if row['amenities'] is not np.nan:
        am = parse_garden_amenity(row['amenities'])
    if row['community_amenities'] is not np.nan:
        cm = parse_garden_amenity(row['community_amenities'])
    if row['Additional Upgraded and Premium Features'] is not np.nan:
        adm = parse_garden_amenity(row['Additional Upgraded and Premium Features'])
    if row['Upgraded Residence Amenities'] is not np.nan:
        ura = parse_garden_amenity(row['Upgraded Residence Amenities'])
    combined_amenities = am + cm + adm + ura
    return set(combined_amenities)

def parse_garden_row(row):
    if row['baths'] is not np.nan:
        row['baths'] = parse_value(row['baths'])
    # If row['beds'] is not null
    if row['beds'] and row['beds'].lower() == 'studio':
        row['beds'] = 1
    else:
        row['beds'] = parse_value(row['beds'])
    if row['price'] is not np.nan:
        temp_price = row['price'].replace(',', '')
        row['price'] = parse_value(temp_price)
    if row['area'] is not np.nan:
        row['area'] = parse_value(row['area'])
    return row

def parse_value(string):
    pattern = r'(\d+)'
    match = re.search(pattern, string)
    if match:
        return int(match.group(1))
    else:
        return None

In [4]:
def process_garden(df):
    usecols = [
        'community_name',
        'community_address',
        'community_mobile',
        'overview_text',
        'floor_plan_name',
        'bath_num',
        'bed_num',
        'price_info',
        'area_info',
        'Apartment Amenities',
        'Community Amenities',
        'Additional Upgraded and Premium Features',
        'Upgraded Residence Amenities',
        'Appliances'
    ]
    df = df[usecols]
    df = df.dropna(axis=0, subset=['bath_num', 'bed_num'])
    df = df.rename(columns={
        'community_name': 'name',
        'community_address': 'address',
        'community_mobile': 'phone',
        'overview_text': 'description',
        'floor_plan_name': 'floor_plan',
        'bath_num': 'baths',
        'bed_num': 'beds',
        'price_info': 'price',
        'area_info': 'area',
        'Apartment Amenities': 'amenities',
        'Community Amenities': 'community_amenities',
    })
    df['combined_amenities'] = df.apply(lambda row: get_amenity_set(row), axis=1)
    df['Appliances'] = (
        df['Appliances']
            .apply(lambda x: set(parse_garden_amenity(x)) if ((x is not np.nan) and (len(x.strip()) > 0)) else np.nan)
    )
    df = df.drop(columns=[
        'amenities',
        'community_amenities',
        'Additional Upgraded and Premium Features',
        'Upgraded Residence Amenities'
    ])
    df = df.apply(lambda row: parse_garden_row(row), axis=1, result_type="expand")
    return df

In [5]:
data_dir = "../data/housing/raw"
data_files = os.listdir(data_dir)
zillows = []
gardens = []
for filename in data_files:
    full_path = os.path.join(data_dir, filename)
    if "zillow" in filename:
        new_df = process_zillow(pd.read_csv(full_path))
        zillows.append(new_df)
    elif "garden" in filename:
        new_df = process_garden(pd.read_csv(full_path))
        gardens.append(new_df)

/var/folders/ml/3qpj1_6x4fj_gcl3ztrny2700000gn/T/ipykernel_16100/2764685260.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['lat', 'long']] = df[["latLong"]].apply(lambda x: extractLatLong(x), axis=1, result_type="expand")
/var/folders/ml/3qpj1_6x4fj_gcl3ztrny2700000gn/T/ipykernel_16100/2764685260.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['lat', 'long']] = df[["latLong"]].apply(lambda x: extractLatLong(x), axis=1, result_type="expand")
/var/folders/ml/3qpj1_6x4fj_gcl3ztrny2700000gn/T

In [6]:
zillow_df = pd.concat(zillows, ignore_index=True)
garden_df = pd.concat(gardens, ignore_index=True)
zillow_df.drop_duplicates(inplace=True)

In [ ]:
garden_df = garden_df.rename(columns={
    'area': 'sqft',
    'Appliances': 'appliances',
    'combined_amenities': 'amenities',
})

In [58]:
zillow_df.to_json("../data/housing/processed/zillow.json", orient="records")
garden_df.to_json("../data/housing/processed/garden.json", orient="records")

In [57]:
zillow_df = zillow_df.rename(columns={'long': 'lon'})
zillow_df.to_json("../data/housing/processed/zillow.json", orient="records")

In [56]:
zillow_df.dtypes

beds         int64
baths      float64
price        int64
address     object
street      object
state       object
city        object
zipcode      int64
lat        float64
lon        float64
dtype: object

In [45]:
garden_df.dtypes

name            object
address         object
phone           object
description     object
floor_plan      object
baths            int64
beds             int64
price          float64
sqft             int64
appliances      object
amenities       object
dtype: object

In [13]:
cleaned_df = pd.read_csv("../data/housing/raw/final_df.csv").drop(columns=['Unnamed: 0', 'url', 'post_id'])
craigslist = cleaned_df[cleaned_df['from'] == 'craigslist'].dropna(axis=1, how='all').reset_index(drop=True)
apartment = cleaned_df[cleaned_df['from'] == 'apartment'].dropna(axis=1, how='all').reset_index(drop=True)

In [27]:
cl_json = pd.read_json('../data/housing/processed/craigslist.json')
cl_json = cl_json.loc[~cl_json['baths'].isin(['shared', 'split'])]
cl_json['baths'] = cl_json['baths'].astype(float)

In [38]:
cl_json['baths'].unique()

array([2. , 1. , 3. , 2.5, 3.5, 1.5, 4.5, 4. ])

In [39]:
cl_json.dtypes

post_id                    int64
beds                       int64
baths                    float64
price                      int64
sqft                       int64
houseType                 object
rentPeriod                object
parking                   object
laundry                   object
pets                      object
smoking                   object
air conditioning          object
wheelchair accessible     object
EV charging               object
furnished                 object
lat                      float64
lon                      float64
dtype: object

In [52]:
cl_json.to_json('../data/housing/processed/craigslist.json', orient='records')

In [60]:
garden = pd.read_csv('../data/housing/raw/garden_community-2022-07-05.csv')
garden.head()

,Unnamed: 0,community_name,community_address,community_mobile,overview_text,floor_plan_name,bath_num,bed_num,price_info,area_info,...,Upgraded Residence Amenities,Appliances,Community,Residence Features,Additional Upgraded Features,Exclusive Upgraded Residence Features,Community Amenities*,Residence Features*,Exclusive Upgraded Residence Features*,parse_date
0,0,360 Luxury Apartments,"9065 Sydney Court, San Diego, CA",(858) 358-7681,Up to First Month Free!* *All prices and spec...,Compass,1 Bath,Studio,"$2,725 - $3,225",662 - 743 sq.ft.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,07/05/2022 07:09:05
1,1,360 Luxury Apartments,"9065 Sydney Court, San Diego, CA",(858) 358-7681,Up to First Month Free!* *All prices and spec...,Escape,1 Bath,1 Bedroom,"$3,135 - $3,610",874 sq.ft.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,07/05/2022 07:09:05
2,2,360 Luxury Apartments,"9065 Sydney Court, San Diego, CA",(858) 358-7681,Up to First Month Free!* *All prices and spec...,Escape with Loft,1 Bath,1 Bedroom,"$3,610 - $3,960",1016 - 1105 sq.ft.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,07/05/2022 07:09:05
3,3,360 Luxury Apartments,"9065 Sydney Court, San Diego, CA",(858) 358-7681,Up to First Month Free!* *All prices and spec...,Evolve,1 Bath,1 Bedroom,"$3,135 - $3,410",880 - 909 sq.ft.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,07/05/2022 07:09:05
4,4,360 Luxury Apartments,"9065 Sydney Court, San Diego, CA",(858) 358-7681,Up to First Month Free!* *All prices and spec...,Evolve with Loft,1 Bath,1 Bedroom,"$3,610",1028 sq.ft.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,07/05/2022 07:09:05


In [62]:
apartment = pd.read_csv('../data/housing/raw/apartment_com-2022-07-05.csv')
apartment.head()

,Option Name,Contact,Address,Size,AvailabilityRent,Monthly Fees,One Time Fees,Transportation,Education,Pet Policy,Distance,Duration,Parking,Amenities,Images,Description
[La Scala Apartments](https://www.apartments.com/la-scala-apartments-san-diego-ca/gpbynnw/),NaN,"[3845 Nobel Dr, San Diego, CA 92122](https://w...","515 - 1,010 sq ft","['Studio, $2,320 – $2,415, Studio, ,, 1 bath, ...",$2320-3610,NaN,* Application Fee: $25\n* Cat Deposit: $250,"Transit / Subway, Distance, Transit / Subway, ...","Colleges & Universities, Distance, Colleges & ...","Cats Allowed, Restrictions:, $250 deposit for ...",NaN,NaN,"Restrictions:, $250 deposit for one cat. $400 ...","Community Amenities, Pool, Fitness Center, Lau...",![Primary Photo - La Scala Apartments](https:/...,About La Scala Apartments\nLa Scala offers lux...
[La Jolla International Gardens](https://www.apartments.com/la-jolla-international-gardens-san-diego-ca/jfps2s6/),NaN,"[3417 Lebon Dr, San Diego, CA 92122](https://w...","652 - 1,215 sq ft","['Florentine, $2,445 – $2,595, 1 bed, ,, 1 bat...",$2445-3655,NaN,* Application Fee: $25\n* Cat Deposit: $250,"Transit / Subway, Distance, Transit / Subway, ...","Colleges & Universities, Distance, Colleges & ...","Cats Allowed, Restrictions:, $250 deposit for ...",NaN,NaN,"Restrictions:, $250 deposit for 1 cat. $400 de...","Community Amenities, Pool, Fitness Center, Lau...",![Primary Photo - La Jolla International Garde...,About La Jolla International Gardens\nAt La Jo...
[Costa Verde Village](https://www.apartments.com/costa-verde-village-san-diego-ca/q2krkky/),NaN,"[8720 Costa Verde Blvd, San Diego, CA 92122](h...","620 - 1,500 sq ft","['Soledad, $2,475 – $2,725, Studio, ,, 1 bath,...",$2475-4950,NaN,* Application Fee: $46,"Transit / Subway, Distance, Transit / Subway, ...","Colleges & Universities, Distance, Colleges & ...","Parking, Garage, 1 space, Assigned Parking, On...",NaN,NaN,"Garage, 1 space, Assigned Parking, One Bedroom...","Community Amenities, Pool, Fitness Center, Ele...",![Primary Photo - Costa Verde Village](https:/...,About Costa Verde Village\nIf you are looking ...
[LUX by Garden](https://www.apartments.com/lux-by-garden-san-diego-ca/2bdptcb/),NaN,"[4200 Brooke Ct, San Diego, CA 92122](https://...","838 - 2,782 sq ft","['1C, $3,665 – $5,650, 1 bed, ,, 1 bath, ,, 90...",$3665-8710,NaN,* Application Fee: $47,"Transit / Subway, Distance, Transit / Subway, ...","Colleges & Universities, Distance, Colleges & ...","Other Fees, Application Fee, $47",NaN,NaN,NaN,"Community Amenities, Pool, Fitness Center, Lau...",![Primary Photo - LUX by Garden](https://image...,About LUX by Garden\nLUX by Garden is the most...
[La Jolla Blue Apartments](https://www.apartments.com/la-jolla-blue-apartments-san-diego-ca/qkdr1yw/),NaN,"[7039 Charmant Dr, San Diego, CA 92122](https:...",687 - 981 sq ft,"['One Bedroom / One Bath, $2,875 – $3,325, 1 b...",$2875-4350,NaN,* Application Fee: $49\n* Cat Deposit: $500\n*...,"Transit / Subway, Distance, Transit / Subway, ...","Colleges & Universities, Distance, Colleges & ...","Dogs Allowed, Restrictions:, We welcome 2 pets...",NaN,NaN,"Restrictions:, We welcome 2 pets per apartment...","Community Amenities, Pool, Fitness Center, Clu...",![Primary Photo - La Jolla Blue Apartments](ht...,About La Jolla Blue Apartments\nWelcome home t...


In [63]:
zillow = pd.read_csv('../data/housing/raw/zillow-2022-07-05.csv')
zillow.head()

,zpid,id,lotId,imgSrc,hasImage,statusType,statusText,detailUrl,latLong,units,...,isUserClaimingOwner,isUserConfirmedClaim,pgapt,sgapt,zestimate,shouldShowZestimateAsPrice,hasVideo,isHomeRec,hasAdditionalAttributions,availabilityDate
0,32.86568--117.235725,32.86568--117.235725,1.001554e+09,https://photos.zillowstatic.com/fp/c16331bf1be...,True,FOR_RENT,Solazzo Apartments Homes,/b/solazzo-apartments-homes-la-jolla-ca-5Xk5YK/,"{'latitude': 32.86568, 'longitude': -117.235725}","[{'price': '$3,165+', 'beds': '1'}]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,32.841976--117.27955,32.841976--117.27955,1.001635e+09,https://photos.zillowstatic.com/fp/025e0ccd6d5...,True,FOR_RENT,Ocean House on Prospect Apartment Homes,/b/ocean-house-on-prospect-apartment-homes-la-...,"{'latitude': 32.841976, 'longitude': -117.27955}","[{'price': '$5,340+', 'beds': '1'}, {'price': ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2066751394,2066751394,NaN,https://photos.zillowstatic.com/fp/69042eebea3...,True,FOR_RENT,Apartment for rent,https://www.zillow.com/homedetails/7623-Eads-A...,"{'latitude': 32.842457, 'longitude': -117.27549}",NaN,...,False,False,ForRent,For Rent,NaN,False,False,False,False,NaN
3,32.841022--117.275635,32.841022--117.275635,NaN,https://photos.zillowstatic.com/fp/f92d733361d...,True,FOR_RENT,For Rent,"/b/La-Jolla-CA/32.841022,-117.275635_ll/","{'latitude': 32.841022, 'longitude': -117.275635}","[{'price': '$5,975+', 'beds': '3'}]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,16857530,16857530,NaN,https://photos.zillowstatic.com/fp/c716e2f7c26...,True,FOR_RENT,Apartment for rent,https://www.zillow.com/homedetails/2205-Camini...,"{'latitude': 32.81866, 'longitude': -117.234184}",NaN,...,False,False,ForRent,For Rent,1304800.0,False,False,False,False,2022-09-01 00:00:00


In [25]:
craigslist = craigslist.loc[~craigslist['num_baths'].isin(['shared', 'split'])]
craigslist = craigslist.drop(columns=['from', 'tf_idf_words'])
craigslist = craigslist.rename(columns={
    'num_baths': 'baths',
    'num_beds': 'beds',
    'price': 'price',
    'area': 'area',
    )

,post_date,num_beds,num_baths,price,address,sqft,features,scrape_date,full_address,houseType,...,parking,laundry,pets,smoking,air conditioning,wheelchair accessible,EV charging,furnished,tfidf_words,from
0,2022-02-05 19:56:55+00:00,3.0,2,3587.0,3889 Midway Drive,1050.0,"cats are OK - purrr, dogs are OK - wooof, apar...",2022-03-27 04:45:44.347396,"3889 Midway Drive,CA",apartment,...,off-street parking,w/d in unit,cats are OK - purrr,NaN,NaN,NaN,NaN,NaN,"['onsite', 'unitsindoor', 'accessprofessional'...",craigslist
1,2022-02-09 16:14:22+00:00,1.0,1,2900.0,7039 Charmant Dr,-1.0,"apartment, w/d in unit, carport",2022-03-27 07:22:10.741597,"7039 Charmant Dr,CA",apartment,...,carport,w/d in unit,NaN,NaN,NaN,NaN,NaN,NaN,"['blue', 'per', '3vd4ntbq', 'fansvaulted', 'st...",craigslist
2,2022-02-09 16:32:09+00:00,1.0,1,2550.0,4956 Cape May Avenue,900.0,"cats are OK - purrr, dogs are OK - wooof, apar...",2022-03-27 04:57:21.315450,"4956 Cape May Avenue,CA",apartment,...,off-street parking,laundry in bldg,cats are OK - purrr,no smoking,NaN,NaN,NaN,NaN,"['cape', 'elan', 'palms', 'pier', 'beach']",craigslist
3,2022-02-09 16:32:48+00:00,2.0,2,2875.0,4975 Del Monte Ave,800.0,"cats are OK - purrr, apartment, laundry on sit...",2022-03-27 05:03:22.265853,"4975 Del Monte Ave,CA",apartment,...,off-street parking,laundry on site,cats are OK - purrr,no smoking,NaN,NaN,NaN,NaN,"['coco', 'palms', 'elan', 'elancocopalms', 'pi...",craigslist
4,2022-02-09 16:37:12+00:00,0.0,1,1745.0,2185 Chatsworth Boulevard,-1.0,"cats are OK - purrr, apartment, laundry on sit...",2022-03-27 07:47:25.414411,"2185 Chatsworth Boulevard,CA",apartment,...,off-street parking,laundry on site,cats are OK - purrr,no smoking,NaN,NaN,NaN,NaN,"['loma', 'highlands', 'elan', 'round', 'needs']",craigslist
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2620,2022-03-27 09:47:43+00:00,2.0,2,1500.0,12557 Ruette Alliante,-1.0,NaN,2022-03-27 04:58:55.887786,"12557 Ruette Alliante,CA",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['ample', 'getting', 'both', 'onepaseo', 'mar']",craigslist
2621,2022-03-27 09:50:40+00:00,1.0,1,1176.0,NaN,-1.0,"cats are OK - purrr, dogs are OK - wooof, furn...",2022-03-27 04:21:36.437093,NaN,apartment,...,attached garage,w/d in unit,cats are OK - purrr,no smoking,NaN,NaN,NaN,furnished,"['laptop', 'udio', 'saving', 'utensils', 'cook...",craigslist
2622,2022-03-27 09:53:56+00:00,1.0,1,1120.0,NaN,-1.0,"cats are OK - purrr, dogs are OK - wooof, furn...",2022-03-27 07:30:38.802107,NaN,apartment,...,attached garage,w/d in unit,cats are OK - purrr,no smoking,NaN,NaN,NaN,furnished,"['bayfront', 'dated', 'lyra', 'caesarstone', '...",craigslist
2623,2022-03-27 10:03:59+00:00,1.0,1,1133.0,NaN,-1.0,"cats are OK - purrr, dogs are OK - wooof, furn...",2022-03-27 06:01:41.005551,NaN,apartment,...,attached garage,w/d in unit,cats are OK - purrr,no smoking,NaN,NaN,NaN,furnished,"['pardee', 'clubs', 'tha', 'phr', 'bosch']",craigslist


In [51]:
apartment_df = pd.read_json("../data/housing/processed/apartment.json")
apartment_df = apartment_df.drop(columns=['from'])
apartment_df.dtypes

beds              float64
baths             float64
price             float64
sqft              float64
name               object
Contact           float64
address            object
One Time Fees      object
Transportation     object
Education          object
Pet Policy         object
Parking            object
Amenities          object
Images             object
description        object
floorplan          object
price_range        object
dtype: object

In [53]:
apartment_df.to_json("../data/housing/processed/apartment.json", orient="records")

In [ ]:
craigslist = pd.read_json("../data/housing/craigslist.json")
garden_df = pd.read_json("../data/housing/garden.json")
zillow_df = pd.read_json("../data/housing/zillow.json")

In [218]:
data_files

['zillow-2022-07-31.csv',
 '.DS_Store',
 'garden_community-2022-05-26.csv',
 'garden_community-2022-05-27.csv',
 'apartment_com-2022-07-31.csv',
 'garden_community-2022-07-31.csv',
 'apartment_com-2022-07-04.csv',
 'garden_community-2022-07-04.csv',
 'garden_community-2022-07-05.csv',
 'apartment_com-2022-07-05.csv',
 'zillow-2022-07-05.csv',
 'zillow-2022-07-04.csv']